In [ ]:
# ===============================================================
# StudyMate - AI PDF Assistant for Google Colab
# ===============================================================
# INSTALLATION CELL - Run this first
# ============================================================================

# Install required packages
!pip install gradio transformers torch PyPDF2 pyttsx3 accelerate gTTS -q

# Install additional dependencies for Colab
!apt-get update -qq
!apt-get install -y espeak espeak-data libespeak1 libespeak-dev -qq
!apt-get install -y festival festvox-kallpc16k -qq

# Import required libraries
import gradio as gr
import PyPDF2
import io
import os
import tempfile
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import threading
import time
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
from gtts import gTTS
from IPython.display import Audio, display

print("✅ All packages installed successfully!")
print("🚀 Starting StudyMate setup...")

# ============================================================================
# STUDYMATE APPLICATION CODE
# ============================================================================

class StudyMate:
    def __init__(self):
        self.tokenizer = None
        self.model = None
        self.pdf_contents = {}  # Store PDF content with filename as key
        self.chat_histories = {}  # Store chat history per PDF
        self.tts_available = False
        self.load_model()
        self.setup_tts()

    def load_model(self):
        """Load IBM Granite model"""
        try:
            print("🔄 Loading IBM Granite model... (This may take a few minutes)")
            self.tokenizer = AutoTokenizer.from_pretrained("ibm-granite/granite-3.0-2b-instruct")
            self.model = AutoModelForCausalLM.from_pretrained(
                "ibm-granite/granite-3.0-2b-instruct",
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                device_map="auto" if torch.cuda.is_available() else None,
                low_cpu_mem_usage=True
            )

            # Move to GPU if available
            if torch.cuda.is_available():
                print("🔥 Using GPU acceleration")
            else:
                print("💻 Using CPU (slower but works)")

            print("✅ IBM Granite model loaded successfully!")
        except Exception as e:
            print(f"❌ Error loading model: {str(e)}")
            print("🔄 Trying alternative model...")
            try:
                # Fallback to a smaller model if main model fails
                self.tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
                self.model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
                print("✅ Fallback model loaded!")
            except:
                print("❌ Model loading failed. Using mock responses.")
                self.tokenizer = None
                self.model = None

    def setup_tts(self):
        """Setup text-to-speech engine for Colab"""
        try:
            # Check if gTTS is available
            from gtts import gTTS
            self.tts_available = True
            print("✅ TTS system ready (using gTTS for Colab)")
        except ImportError:
            self.tts_available = False
            print("⚠ gTTS not installed. Audio playback will be simulated.")
        except Exception as e:
            print(f"⚠ TTS setup info: {str(e)}")
            self.tts_available = False

    def extract_pdf_text(self, pdf_file):
        """Extract text from uploaded PDF"""
        try:
            if pdf_file is None:
                return None, "❌ No PDF file provided"

            # Handle file path (Colab returns file path as string)
            if isinstance(pdf_file, str):
                with open(pdf_file, 'rb') as file:
                    pdf_reader = PyPDF2.PdfReader(file)
                    text = ""

                    for page_num, page in enumerate(pdf_reader.pages):
                        try:
                            text += f"\n--- Page {page_num + 1} ---\n"
                            text += page.extract_text()
                        except Exception as e:
                            print(f"Error reading page {page_num + 1}: {str(e)}")
                            continue
            else:
                # Handle file object
                pdf_reader = PyPDF2.PdfReader(pdf_file)
                text = ""

                for page_num, page in enumerate(pdf_reader.pages):
                    try:
                        text += f"\n--- Page {page_num + 1} ---\n"
                        text += page.extract_text()
                    except Exception as e:
                            print(f"Error reading page {page_num + 1}: {str(e)}")
                            continue

            if not text.strip():
                return None, "❌ Could not extract text from PDF. The PDF might be image-based or corrupted."

            filename = os.path.basename(pdf_file if isinstance(pdf_file, str) else pdf_file.name)
            self.pdf_contents[filename] = text
            self.chat_histories[filename] = []

            return filename, f"✅ Successfully loaded PDF: {filename}\n📄 Extracted {len(text)} characters from {len(pdf_reader.pages)} pages"

        except Exception as e:
            return None, f"❌ Error processing PDF: {str(e)}"

    def format_answer_with_bullets(self, answer):
        """Convert answer to bullet points with emojis"""
        if not answer:
            return "❌ No answer generated"

        # Split answer into sentences
        sentences = [s.strip() for s in answer.replace('\n', ' ').split('.') if s.strip()]

        if not sentences:
            return "📝 " + answer

        # Emojis for bullet points
        emojis = ["📌", "✨", "💡", "🔍", "📚", "⭐", "🎯", "💭", "🔥", "🌟"]

        formatted_answer = ""
        for i, sentence in enumerate(sentences[:8]):  # Limit to 8 points
            emoji = emojis[i % len(emojis)]
            if sentence.strip():
                formatted_answer += f"{emoji} {sentence.strip()}.\n\n"

        return formatted_answer.strip()

    def generate_answer(self, question, pdf_filename):
        """Generate answer using IBM Granite model"""
        if not pdf_filename or pdf_filename not in self.pdf_contents:
            return "❌ Please upload a PDF first!"

        if not question.strip():
            return "❌ Please ask a question!"

        pdf_content = self.pdf_contents[pdf_filename]

        # If model is not available, provide an enhanced mock response
        if not self.model or not self.tokenizer:
            # Create a more intelligent mock response based on PDF content
            keywords = question.lower().split()
            relevant_sentences = []

            for sentence in pdf_content.split('.'):
                if any(keyword in sentence.lower() for keyword in keywords):
                    relevant_sentences.append(sentence.strip())
                    if len(relevant_sentences) >= 3:
                        break

            if relevant_sentences:
                mock_answer = f"Based on your PDF content, here are the key points about '{question}': " + " ".join(relevant_sentences[:2])
            else:
                mock_answer = f"Your question about '{question}' relates to the content in your PDF. The document contains {len(pdf_content)} characters of information that may help answer your query. Please try different keywords or rephrase your question."

            return self.format_answer_with_bullets(mock_answer)

        try:
            # Prepare context with PDF content (truncated to avoid token limits)
            context = pdf_content[:1500]  # Limit context size for better performance

            # Create a focused prompt
            prompt = f"""Document: {context}

Question: {question}

Based on the document above, provide a clear and concise answer:"""

            # Tokenize input
            inputs = self.tokenizer.encode(prompt, return_tensors="pt", max_length=1024, truncation=True)

            # Move to appropriate device
            if torch.cuda.is_available():
                inputs = inputs.to(self.model.device)

            # Generate response
            with torch.no_grad():
                outputs = self.model.generate(
                    inputs,
                    max_new_tokens=150,
                    temperature=0.7,
                    do_sample=True,
                    pad_token_id=self.tokenizer.eos_token_id,
                    repetition_penalty=1.1
                )

            # Decode the response
            full_response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Extract only the new generated part
            response = full_response[len(prompt):].strip()

            if not response:
                response = "I found relevant information in your PDF, but need more context to provide a specific answer. Try rephrasing your question."

            # Format as bullet points
            formatted_response = self.format_answer_with_bullets(response)

            return formatted_response

        except Exception as e:
            print(f"Generation error: {str(e)}")
            return f"❌ Error generating answer: {str(e)}\n\n💡 Please try a simpler question or check if the PDF contains relevant information."

    def speak_answer(self, text):
        """Convert text to speech using gTTS and display in Colab"""
        if not self.tts_available:
            return "⚠ Audio playback not available. gTTS library not found."

        try:
            # Clean the text for TTS
            clean_text = ''.join(char for char in text if char.isalnum() or char.isspace() or char in '.,!?')

            if not clean_text.strip():
                return "⚠ No text to speak"

            # Create gTTS object
            tts = gTTS(text=clean_text, lang='en')

            # Save the audio to a temporary file
            with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as fp:
                temp_filename = fp.name
                tts.save(temp_filename)

            # Display the audio player in Colab
            display(Audio(temp_filename, autoplay=True))

            # Clean up the temporary file after playback (or later)
            # For simplicity in this example, we just return a status message.
            # In a real app, you might manage these temp files more robustly.

            return f"🔊 Playing audio... (Length: {len(clean_text)} characters)"

        except Exception as e:
            print(f"TTS Error: {str(e)}")
            return f"❌ TTS Error: {str(e)}"

    def get_chat_history(self, pdf_filename):
        """Get chat history for a specific PDF"""
        if pdf_filename and pdf_filename in self.chat_histories:
            return self.chat_histories[pdf_filename]
        return []

    def add_to_history(self, pdf_filename, question, answer):
        """Add Q&A to chat history"""
        if pdf_filename and pdf_filename in self.chat_histories:
            timestamp = datetime.now().strftime("%H:%M:%S")
            self.chat_histories[pdf_filename].append({
                "time": timestamp,
                "question": question,
                "answer": answer
            })

    def remove_pdf(self, pdf_filename):
        """Remove PDF and its history"""
        if pdf_filename:
            if pdf_filename in self.pdf_contents:
                del self.pdf_contents[pdf_filename]
            if pdf_filename in self.chat_histories:
                del self.chat_histories[pdf_filename]
            return f"✅ Removed PDF: {pdf_filename} and its chat history"
        return "❌ No PDF to remove"

    def get_available_pdfs(self):
        """Get list of available PDFs"""
        return list(self.pdf_contents.keys())

# Initialize StudyMate
print("🎓 Initializing StudyMate...")
study_mate = StudyMate()

def process_pdf(pdf_file):
    """Process uploaded PDF"""
    filename, message = study_mate.extract_pdf_text(pdf_file)
    available_pdfs = study_mate.get_available_pdfs()
    return message, gr.update(choices=available_pdfs, value=filename if filename else None), gr.update(value="")

def ask_question(question, selected_pdf):
    """Process question and generate answer"""
    if not selected_pdf:
        return "❌ Please select a PDF first!", "", gr.update(value="")

    print(f"🤔 Processing question: {question}")
    answer = study_mate.generate_answer(question, selected_pdf)
    study_mate.add_to_history(selected_pdf, question, answer)

    # Format chat history
    history = study_mate.get_chat_history(selected_pdf)
    history_text = ""
    for entry in history[-10:]:  # Show last 10 Q&As
        history_text += f"🕒 {entry['time']}\n❓ *Q:* {entry['question']}\n💬 *A:* {entry['answer']}\n\n" + "="*50 + "\n\n"

    return answer, history_text, gr.update(value="")

def play_audio(answer_text):
    """Play audio of the answer"""
    if not answer_text:
        return "❌ No answer to play"
    return study_mate.speak_answer(answer_text)

def remove_selected_pdf(selected_pdf):
    """Remove selected PDF"""
    message = study_mate.remove_pdf(selected_pdf)
    available_pdfs = study_mate.get_available_pdfs()
    return message, gr.update(choices=available_pdfs, value=None), "", ""

def get_sample_questions(selected_pdf):
    """Generate sample questions based on PDF content"""
    if not selected_pdf or selected_pdf not in study_mate.pdf_contents:
        return "Upload a PDF first to see sample questions!"

    sample_questions = [
        "❓ What is the main topic of this document?",
        "❓ Can you summarize the key points?",
        "❓ What are the important definitions mentioned?",
        "❓ Are there any examples provided?",
        "❓ What conclusions can be drawn?",
        "❓ What methodology is used?",
        "❓ Are there any recommendations?",
        "❓ What are the findings?"
    ]

    return "\n".join(sample_questions)

# ============================================================================
# GRADIO INTERFACE
# ============================================================================

# Custom CSS for better appearance
custom_css = """
.gradio-container {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
    font-family: 'Arial', sans-serif;
}

.main-header {
    text-align: center;
    color: white !important;
    font-size: 3em !important;
    font-weight: bold !important;
    margin: 20px 0 !important;
    text-shadow: 2px 2px 4px rgba(0,0,0,0.5) !important;
    background: linear-gradient(45deg, #ff6b6b, #4ecdc4) !important;
    -webkit-background-clip: text !important;
    -webkit-text-fill-color: transparent !important;
    background-clip: text !important;
}

.description {
    text-align: center;
    color: white !important;
    font-size: 1.3em !important;
    margin-bottom: 30px !important;
    text-shadow: 1px 1px 2px rgba(0,0,0,0.3) !important;
}

.footer {
    text-align: center;
    margin-top: 30px;
    color: white !important;
    font-size: 1.1em !important;
    text-shadow: 1px 1px 2px rgba(0,0,0,0.3) !important;
}

.section-header {
    color: #2d3748 !important;
    font-weight: bold !important;
    font-size: 1.2em !important;
    margin: 10px 0 !important;
}

.gr-button {
    transition: all 0.3s ease !important;
}

.gr-button:hover {
    transform: translateY(-2px) !important;
    box-shadow: 0 4px 8px rgba(0,0,0,0.2) !important;
}
"""

# Create the main interface
with gr.Blocks(
    theme=gr.themes.Soft(
        primary_hue="blue",
        secondary_hue="cyan",
        neutral_hue="slate"
    ).set(
        body_background_fill="linear-gradient(135deg, #667eea 0%, #764ba2 100%)",
        block_background_fill="rgba(255, 255, 255, 0.9)"
    ),
    css=custom_css,
    title="🎓 StudyMate - AI PDF Assistant"
) as app:

    # Header
    gr.HTML("""
    <div class="main-header">
        🎓 StudyMate - AI PDF Assistant 📚
    </div>
    <div class="description">
        🚀 Upload PDFs • 💭 Ask Questions • 🎧 Get Audio Answers • 📖 Learn Smarter!
        <br>
        ✨ Powered by IBM Granite AI Model ✨
    </div>
    """)

    # Main content area
    with gr.Row():
        # Left column - PDF management
        with gr.Column(scale=1):
            gr.HTML('<div class="section-header">📤 PDF Management</div>')

            pdf_upload = gr.File(
                label="📁 Upload Your PDF",
                file_types=[".pdf"],
                type="filepath",
                elem_id="pdf_upload"
            )

            pdf_status = gr.Textbox(
                label="📊 Status",
                interactive=False,
                lines=4,
                placeholder="Upload a PDF to get started..."
            )

            selected_pdf = gr.Dropdown(
                label="📋 Active PDF",
                choices=[],
                interactive=True,
                info="Select which PDF to query"
            )

            with gr.Row():
                remove_btn = gr.Button("🗑 Remove PDF", variant="secondary", size="sm")
                refresh_btn = gr.Button("🔄 Refresh", variant="secondary", size="sm")

            # Sample questions
            sample_questions = gr.Textbox(
                label="💡 Sample Questions",
                lines=8,
                interactive=False,
                placeholder="Upload a PDF to see sample questions..."
            )

        # Right column - Q&A interface
        with gr.Column(scale=2):
            gr.HTML('<div class="section-header">💭 Ask Questions</div>')

            question_input = gr.Textbox(
                label="❓ Your Question",
                placeholder="What would you like to know about your PDF?",
                lines=3,
                info="Ask anything about the content in your uploaded PDF"
            )

            with gr.Row():
                ask_btn = gr.Button("🔍 Get Answer", variant="primary", size="lg")
                clear_btn = gr.Button("🧹 Clear", variant="secondary", size="sm")

            answer_output = gr.Textbox(
                label="💬 AI Answer",
                interactive=False,
                lines=10,
                placeholder="Your answer will appear here..."
            )

            with gr.Row():
                audio_btn = gr.Button("🔊 Play Audio", variant="secondary")
                copy_btn = gr.Button("📋 Copy Answer", variant="secondary")

            audio_status = gr.Textbox(
                label="🎵 Audio Status",
                interactive=False,
                lines=2,
                show_label=False
            )

    # Bottom section - Chat history
    with gr.Row():
        with gr.Column():
            gr.HTML('<div class="section-header">📜 Chat History</div>')
            history_output = gr.Textbox(
                label="💭 Previous Questions & Answers",
                interactive=False,
                lines=12,
                placeholder="Your Q&A history will appear here...",
                show_label=False
            )

    # Footer
    gr.HTML("""
    <div class="footer">
        <p>🎯 <strong>StudyMate Features:</strong></p>
        <p>📚 Multi-PDF Support • 💡 Smart Q&A • 🎧 Audio Answers • 📝 Bullet Points • 💾 Chat History</p>
        <p>🚀 <strong>Made for Google Colab</strong> • Powered by IBM Granite AI 🧠</p>
        <p>💡 <strong>Tip:</strong> Ask specific questions for better answers!</p>
    </div>
    """)

    # Event handlers
    pdf_upload.change(
        process_pdf,
        inputs=[pdf_upload],
        outputs=[pdf_status, selected_pdf, history_output]
    )

    selected_pdf.change(
        get_sample_questions,
        inputs=[selected_pdf],
        outputs=[sample_questions]
    )

    ask_btn.click(
        ask_question,
        inputs=[question_input, selected_pdf],
        outputs=[answer_output, history_output, question_input]
    )

    question_input.submit(
        ask_question,
        inputs=[question_input, selected_pdf],
        outputs=[answer_output, history_output, question_input]
    )

    audio_btn.click(
        play_audio,
        inputs=[answer_output],
        outputs=[audio_status]
    )

    remove_btn.click(
        remove_selected_pdf,
        inputs=[selected_pdf],
        outputs=[pdf_status, selected_pdf, answer_output, history_output]
    )

    clear_btn.click(
        lambda: ("", ""),
        outputs=[question_input, answer_output]
    )

    refresh_btn.click(
        lambda: study_mate.get_available_pdfs(),
        outputs=[selected_pdf]
    )

# ============================================================================
# LAUNCH APPLICATION
# ============================================================================

if __name__ == "__main__":
    print("\n" + "="*60)
    print("🎓 STUDYMATE - AI PDF ASSISTANT")
    print("="*60)
    print("✅ Setup complete!")
    print("📱 Launching web interface...")
    print("🌐 You'll get a public URL to access your app")
    print("📚 Upload PDFs and start learning!")
    print("="*60 + "\n")

    # Launch with Colab-optimized settings
    app.launch(
        share=True,          # Create public URL
        debug=True,          # Show errors
        server_name="0.0.0.0",  # Allow external access
        show_error=True,     # Display errors in interface
        quiet=False          # Show launch logs
    )

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 11.3 MB/s eta 0:00:00
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 126435 files and directories currently installed.)
Preparing to unpack .../0-libportaudio2_19.6.0-1.1_amd64.deb ...
Unpacking libportaudio2:amd64 (19.6.0-1.1) ...
Selecting previously unselected package libsonic0:amd64.
Preparing to unpack .../1-libsonic0_0.2.0-11build1_amd64.deb ...
Unpacking libsonic0:amd64 (0.2.0-11build1) ...
Selecting previously unselected package espeak-data:amd64.
Preparing to unpack .../2-espeak-data_1.48.15+dfsg-3_amd64.deb ...
Unpacking espeak-data:amd64 (1.48.15+dfsg-3) ...
Selecting previously unselected package libespeak1:amd64.
Prep

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

🔥 Using GPU acceleration
✅ IBM Granite model loaded successfully!
✅ TTS system ready (using gTTS for Colab)

🎓 STUDYMATE - AI PDF ASSISTANT
✅ Setup complete!
📱 Launching web interface...
🌐 You'll get a public URL to access your app
📚 Upload PDFs and start learning!

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e730f15a3276b6ccac.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


🤔 Processing question: types of dc machines
